<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EA%B3%BC%EC%A0%95/(%EB%A7%9D%ED%95%A8)%EC%A6%9D%EB%B6%84%EB%90%9C_G_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%99%EC%8A%B5_inc2_testsize0_01_epoch50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 1.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595746 sha256=43ca5638ecfc453700ae9b21cf87adade39f006d0d60e922447268425ad79e4e
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 769 kB 5.0 MB/s 
     |████████████████████████████████| 3.0 MB 46.2 MB/s 
     |████████████████████████████████| 895 kB 46.7 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-eaamx2lx
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/df_G_inc.csv')

## 데이터 전처리

In [4]:
df['digit_3'].value_counts()

474    36453
478    30566
452    28176
477    27462
471    26189
472    25668
464    18453
463    17873
462    15510
465    14652
468    14574
461    13968
451    13608
475    13031
467    12096
453     9891
473     9312
479     9218
466     8154
476     7188
Name: digit_3, dtype: int64

In [5]:
df.loc[(df['digit_3'] == 451), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 452), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 453), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 461), 'digit_3'] = 3  
df.loc[(df['digit_3'] == 462), 'digit_3'] = 4  
df.loc[(df['digit_3'] == 463), 'digit_3'] = 5  
df.loc[(df['digit_3'] == 464), 'digit_3'] = 6
df.loc[(df['digit_3'] == 465), 'digit_3'] = 7  
df.loc[(df['digit_3'] == 466), 'digit_3'] = 8  
df.loc[(df['digit_3'] == 467), 'digit_3'] = 9  
df.loc[(df['digit_3'] == 468), 'digit_3'] = 10  
df.loc[(df['digit_3'] == 471), 'digit_3'] = 11  
df.loc[(df['digit_3'] == 472), 'digit_3'] = 12  
df.loc[(df['digit_3'] == 473), 'digit_3'] = 13
df.loc[(df['digit_3'] == 474), 'digit_3'] = 14  
df.loc[(df['digit_3'] == 475), 'digit_3'] = 15  
df.loc[(df['digit_3'] == 476), 'digit_3'] = 16  
df.loc[(df['digit_3'] == 477), 'digit_3'] = 17  
df.loc[(df['digit_3'] == 478), 'digit_3'] = 18  
df.loc[(df['digit_3'] == 479), 'digit_3'] = 19

In [6]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['매장에서 일반인 대상으로 의류소매', '14'], ['매장에서 일반인대상으로 의류소매', '14'], ['사업장에서 일반인대상 의류판매', '14'], ['일반고객에게 신발매장에서 일반인을 대상으로 신발', '14'], ['일반소비자 가정용 직물제품소매점 커튼 방석 침구류등 소매', '14']]


## Train data & test data

In [7]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.01, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [8]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [9]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [10]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [11]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [12]:
data_train[0]

(array([   2, 1998, 6903, 3807, 5440, 5816, 3312, 6379,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(9, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 14)

In [13]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [14]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=20,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [15]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/5446 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 3.014875888824463 train acc 0.046875
epoch 1 batch id 201 loss 3.007012367248535 train acc 0.054881840796019904
epoch 1 batch id 401 loss 2.9464871883392334 train acc 0.0633182668329177
epoch 1 batch id 601 loss 2.9399025440216064 train acc 0.08192075707154742
epoch 1 batch id 801 loss 2.6959240436553955 train acc 0.11290574282147316
epoch 1 batch id 1001 loss 2.339674949645996 train acc 0.15289398101898102
epoch 1 batch id 1201 loss 2.108677864074707 train acc 0.19501977518734387
epoch 1 batch id 1401 loss 1.9016649723052979 train acc 0.24257226980728053
epoch 1 batch id 1601 loss 1.390313744544983 train acc 0.2920537945034353
epoch 1 batch id 1801 loss 1.373573899269104 train acc 0.3389783453636868
epoch 1 batch id 2001 loss 1.3276872634887695 train acc 0.38179347826086957
epoch 1 batch id 2201 loss 0.9238194823265076 train acc 0.4194755224897774
epoch 1 batch id 2401 loss 0.8969098925590515 train acc 0.45346990837151185
epoch 1 batch id 2601 loss 0.6534993648

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 11 batch id 1 loss 0.20117568969726562 train acc 0.953125
epoch 11 batch id 201 loss 0.21627183258533478 train acc 0.9583333333333334
epoch 11 batch id 401 loss 0.0700015276670456 train acc 0.9563980673316709
epoch 11 batch id 601 loss 0.28470292687416077 train acc 0.9559588186356073
epoch 11 batch id 801 loss 0.19133532047271729 train acc 0.9568898252184769
epoch 11 batch id 1001 loss 0.13884298503398895 train acc 0.9570741758241759
epoch 11 batch id 1201 loss 0.27234840393066406 train acc 0.9570019775187344
epoch 11 batch id 1401 loss 0.2699441909790039 train acc 0.9572292112776588
epoch 11 batch id 1601 loss 0.17891080677509308 train acc 0.9573801530293566
epoch 11 batch id 1801 loss 0.26417842507362366 train acc 0.9576884369794558
epoch 11 batch id 2001 loss 0.23765693604946136 train acc 0.9577711144427786
epoch 11 batch id 2201 loss 0.1435946375131607 train acc 0.9577890731485689
epoch 11 batch id 2401 loss 0.13045568764209747 train acc 0.9576803935860059
epoch 11 batch id 2

  0%|          | 0/56 [00:00<?, ?it/s]

epoch 11 test acc 0.9221540178571429


  0%|          | 0/5446 [00:00<?, ?it/s]

epoch 12 batch id 1 loss 0.03575396537780762 train acc 0.96875
epoch 12 batch id 201 loss 0.28359219431877136 train acc 0.959110696517413
epoch 12 batch id 401 loss 0.10376963019371033 train acc 0.9591645885286783
epoch 12 batch id 601 loss 0.5021708607673645 train acc 0.9589746256239601
epoch 12 batch id 801 loss 0.1684217005968094 train acc 0.9591526217228464
epoch 12 batch id 1001 loss 0.10780642181634903 train acc 0.9597746003996004
epoch 12 batch id 1201 loss 0.14470309019088745 train acc 0.9596169858451291
epoch 12 batch id 1401 loss 0.2655213177204132 train acc 0.9594263026409707
epoch 12 batch id 1601 loss 0.1961902230978012 train acc 0.9594394128669581
epoch 12 batch id 1801 loss 0.1847773790359497 train acc 0.9595884230982787
epoch 12 batch id 2001 loss 0.21333527565002441 train acc 0.959364067966017
epoch 12 batch id 2201 loss 0.10061696171760559 train acc 0.9592727737392095
epoch 12 batch id 2401 loss 0.09883254766464233 train acc 0.9593723969179508
epoch 12 batch id 2601 l

In [ ]:
#학습 모델 저장
torch.save(model, 'G_inc_model_2.pt')